In [98]:
%matplotlib qt
import os
import pathlib
from time import time

import matplotlib.pyplot as plt

from fmm import Fmm
from fmm.kernel import laplace_p2p_serial

# Plotting parameters
plt.rc('font', family='serif', serif='Times')
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)
plt.rc('axes', labelsize=8)

# Dimensions for column plots
width = 3.487
height = width / 1.618

HERE = pathlib.Path(os.getcwd())
FIGURE_SAVEPATH = os.path.abspath(HERE.parent.parent / 'article/figures')

# The Impact of Choice of Compression Parameter $K$

## Generate Data

Uncomment to generate data

In [58]:
# ! fmm generate-test-data -c K_1 && fmm compute-operators -c K_1
# ! fmm generate-test-data -c K_5 && fmm compute-operators -c K_5
# ! fmm generate-test-data -c K_10 && fmm compute-operators -c K_10
# ! fmm generate-test-data -c K_15 && fmm compute-operators -c K_15
# ! fmm generate-test-data -c K_20 && fmm compute-operators -c K_20
# ! fmm generate-test-data -c K_25 && fmm compute-operators -c K_25
# ! fmm generate-test-data -c K_30 && fmm compute-operators -c K_30
# ! fmm generate-test-data -c K_35 && fmm compute-operators -c K_35
# ! fmm generate-test-data -c K_40 && fmm compute-operators -c K_40
# ! fmm generate-test-data -c K_45 && fmm compute-operators -c K_45
# ! fmm generate-test-data -c K_50 && fmm compute-operators -c K_50

## Accuracy

How does compression parameter effect accuracy? We expect $p$ digits of accuracy from theory, where $p$ is the order of the multipole/local expansions

In [60]:
# Number of runs
nruns = 3

# Container for timings
tvec = [[] for n in range(nruns)]


# Compression parameters tested
Kvec = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

# Evaluate FMM for each experiment
for i in range(nruns):
    # Reinitialise experiments
    evec = [Fmm(f'K_{K}') for K in Kvec]
    for e in evec:
        start = time()
        e.run()
        tvec[i].append(time()-start)

In [61]:
# Results of direct computation for each experiment
d = laplace_p2p_serial(evec[0].sources, evec[0].targets, evec[0].source_densities)

In [62]:
relerrvec = [abs(d-e.target_potentials)/d for e in evec]

In [63]:
relerrmean = [np.mean(err) for err in relerrvec]
relerrstd = [np.std(err) for err in relerrvec]

In [101]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

ax.semilogy(Kvec, relerrmean, '--o', ms=3, lw=0.4, c='0.25')
ax.set_ylabel('Mean Relative Error ($\epsilon_{\ rel}$)')
ax.set_xlabel('Compression Rank ($K$)')
ax.set_xticks(Kvec)

fig.set_size_inches(width, height)
fp = FIGURE_SAVEPATH  + '/compression_accuracy.pdf'
plt.savefig(fp)
plt.show()

## Runtimes

How does compression parameter effect runtime?

In [71]:
tvec = np.array(tvec)
tvecmean = np.mean(tvec, axis=0)

In [100]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

ax.plot(Kvec, tvecmean, '--o', ms=3, lw=0.4, c='0.25')
ax.set_ylabel('FMM Runtime ($s$)')
ax.set_xlabel('Compression Rank ($K$)')
ax.set_xticks(Kvec)

fig.set_size_inches(width, height)
fp = FIGURE_SAVEPATH  + '/compression_runtime.pdf'
plt.savefig(fp)
plt.show()